## Contents

### 1) Pre-liminary Experiment
- in order to have proper values for epsilons and hyper-volume calculation

### 2) MORDM under the Worst Scenario
- Identify 16 solutions
- Filter out 7 solutions, which Number of Deaths exceed the standard from the literature

### 3) Possible Thresholds for Costs
- Deaths: 0, literature and proportion
- Costs: maximum, mean and proportion

In [1]:
import re, time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from ema_workbench import (Model, MultiprocessingEvaluator, ScalarOutcome, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume, ArchiveLogger
from ema_workbench.analysis import parcoords

from ema_workbench.util import ema_logging

from problem_formulation import get_model_for_problem_formulation
ema_logging.log_to_stderr(ema_logging.INFO)

C:\Anaconda3\envs\py36_32\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Anaconda3\envs\py36_32\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


<Logger EMA (DEBUG)>

In [2]:
dike_model = get_model_for_problem_formulation(5)

[MainProcess/INFO] model initialized


In [3]:
worst = pd.read_csv("2_1.csv", index_col=0).iloc[:, :-3].T.apply(lambda x: np.double(re.sub(r"\'\,.*|.*\(\'", '', x[0])) if type(x[0]) == str else x[0], axis=1)
worst = worst.to_dict()
worst

{'A.1_Bmax': 174.26331739472218,
 'A.1_pfail': 0.3377237967993181,
 'A.1_Brate': 1000.0,
 'A.2_Bmax': 124.0203965185816,
 'A.2_pfail': 9.396791780028753e-05,
 'A.2_Brate': 1000.0,
 'A.3_Bmax': 141.34225786212474,
 'A.3_pfail': 0.043896056641610254,
 'A.3_Brate': 1.5,
 'A.4_Bmax': 40.92167903850999,
 'A.4_pfail': 0.008756681981009297,
 'A.4_Brate': 1000.0,
 'A.5_Bmax': 34.057917745757486,
 'A.5_pfail': 1.3044916758325106e-05,
 'A.5_Brate': 1.5,
 'discount rate': 2.5,
 'A.0_ID flood wave shape': 123.0}

In [4]:
best = pd.read_csv("2_2.csv", index_col=0).iloc[:, :-3].T.apply(lambda x: np.double(re.sub(r"\'\,.*|.*\(\'", '', x[0])) if type(x[0]) == str else x[0], axis=1)
best = best.to_dict()
best

{'A.1_Bmax': 179.30868903488053,
 'A.1_pfail': 0.9887005191150104,
 'A.1_Brate': 1000.0,
 'A.2_Bmax': 154.87267584911348,
 'A.2_pfail': 0.9998372248805868,
 'A.2_Brate': 1.5,
 'A.3_Bmax': 273.5596600903342,
 'A.3_pfail': 0.9998028817287072,
 'A.3_Brate': 1000.0,
 'A.4_Bmax': 112.36314107288212,
 'A.4_pfail': 0.9364865985102534,
 'A.4_Brate': 1000.0,
 'A.5_Bmax': 149.6243667646259,
 'A.5_pfail': 0.9999611199339804,
 'A.5_Brate': 1.5,
 'discount rate': 1.5,
 'A.0_ID flood wave shape': 129.0}

In [5]:
b_scenario = Scenario("best", **best)
w_scenario = Scenario("worst", **worst)

policy = {'DikeIncrease': 0, 'DaysToThreat': 0, 'RfR': 0}
pol0 = {}
for key in dike_model.levers:
    s1, s2 = key.name.split('_')
    pol0.update({key.name: policy[s2]})                    
policy0 = Policy('Policy 0', **pol0)
pol0

{'A.1_DikeIncrease': 0,
 'A.2_DikeIncrease': 0,
 'A.3_DikeIncrease': 0,
 'A.4_DikeIncrease': 0,
 'A.5_DikeIncrease': 0,
 '0_RfR': 0,
 '1_RfR': 0,
 '2_RfR': 0,
 '3_RfR': 0,
 '4_RfR': 0,
 'EWS_DaysToThreat': 0}

### Test Run

In [6]:
# best
ref_scenario = b_scenario
start = time.time()
dike_model.run_model(ref_scenario, policy0)
end = time.time()
print(end - start)
print("best", dike_model.outcomes_output)

# worst
ref_scenario = w_scenario
start = time.time()
dike_model.run_model(ref_scenario, policy0)
end = time.time()
print(end - start)
print("worst", dike_model.outcomes_output)

1.174856424331665
best {'Expected Number of Deaths(1/3)': 0.0, 'Expected Number of Deaths(2/5)': 0.0, 'Expected Number of Deaths(4)': 4.91885090174396e-05, 'Dike Investment Costs(1/3)': 0, 'Dike Investment Costs(2/5)': 0, 'Dike Investment Costs(4)': 0, 'RfR Total Costs': 0.0, 'Expected Evacuation Costs': 0.0}
1.4890155792236328
worst {'Expected Number of Deaths(1/3)': 1.0722675192406605, 'Expected Number of Deaths(2/5)': 0.4456688504275548, 'Expected Number of Deaths(4)': 0.020212905547828, 'Dike Investment Costs(1/3)': 0, 'Dike Investment Costs(2/5)': 0, 'Dike Investment Costs(4)': 0, 'RfR Total Costs': 0.0, 'Expected Evacuation Costs': 0.0}


### 1) Pre-liminary Experiment
- in order to have proper values for epsilons and hyper-volume calculation

In [9]:
n_scenarios = 1
n_policies = 4000
start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies = n_policies)
    end = time.time()
print(end - start, "secs")

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1 scenarios * 4000 policies * 1 model(s) = 4000 experiments
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] 2400 cases completed
[MainProcess/INFO] 2800 cases completed
[MainProcess/INFO] 3200 cases completed
[MainProcess/INFO] 3600 cases completed
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


1692.7042639255524 secs


In [7]:
epsilon = pd.DataFrame(results[1]).max().values/1000
hyper_min = [0,]*len(dike_model.outcomes)
hyper_max = pd.DataFrame(results[1]).max().values * 2

### 2) MORDM under the Worst Scenario
- Identify 16 solutions
- Filter out 7 solutions, which Number of Deaths exceed the standard from the literature

In [8]:
decision_varnames=list(dike_model.levers.keys())
outcome_varnames=list(dike_model.outcomes.keys())

nfe = 5000

In [16]:
ref_scenario = w_scenario
convergence_metrics = [
#     EpsilonProgress(),
#     HyperVolume(minimum=hyper_min, maximum=hyper_max),
    ArchiveLogger("./archive/worst", decision_varnames=decision_varnames, outcome_varnames=outcome_varnames)
]


start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(nfe = nfe, searchover = 'levers', 
                                              epsilons = epsilon, convergence = convergence_metrics, 
                                              reference = ref_scenario)
end = time.time()
print("Time:", round(end-start)/60, "Minutes")

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 100 policies * 1 model(s) = 100 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 1: 100/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 100 policies * 1 model(s) = 100 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 2: 200/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 100 policies * 1 model(s) = 100 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 3: 300/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 100 policies * 1

[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 29: 2886/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 99 policies * 1 model(s) = 99 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 30: 2985/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 100 policies * 1 model(s) = 100 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] generation 31: 3085/5000 nfe
[MainProcess/INFO] performing 1 scenarios * 99 policies * 1 model(s) = 99 experiments
[MainProcess/INFO] 33 cases completed
[MainProcess/INFO] 66 cases completed
[MainProcess/INFO] 99 cases completed
[MainProcess/INFO] e

Time: 21.683333333333334 Minutes


In [9]:
from os import listdir
from os.path import isfile, join

mypath = "./archive/best"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

def func(x):
    digit = re.sub(r".*_|\..*", "", x)
    if len(digit) == 1:
        digit = "".join(["0",digit])
        x = re.sub(r"\d", digit, x)
    return x
vfunc = np.vectorize(func)
onlyfiles = vfunc(onlyfiles)
onlyfiles.sort()

best_results = pd.read_csv("".join([mypath, "/", onlyfiles[-1]]), index_col=0)

mypath = "./archive/worst"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = vfunc(onlyfiles)
onlyfiles.sort()

worst_results = pd.read_csv("".join([mypath, "/", onlyfiles[-1]]), index_col=0)

In [10]:
dike_model.outcomes.keys()

odict_keys(['Expected Number of Deaths(1/3)', 'Expected Number of Deaths(2/5)', 'Expected Number of Deaths(4)', 'Dike Investment Costs(1/3)', 'Dike Investment Costs(2/5)', 'Dike Investment Costs(4)', 'RfR Total Costs', 'Expected Evacuation Costs'])

#### 16 Solutions from MORDM under the Worst Scenario

In [11]:
data = worst_results.loc[:, dike_model.outcomes.keys()]
data

,Expected Number of Deaths(1/3),Expected Number of Deaths(2/5),Expected Number of Deaths(4),Dike Investment Costs(1/3),Dike Investment Costs(2/5),Dike Investment Costs(4),RfR Total Costs,Expected Evacuation Costs
0,0.608345,0.231049,0.000555,0.000000e+00,0.000000e+00,1.340924e+07,0.0,42919.029867
1,0.000000,0.641804,0.024754,8.122873e+07,0.000000e+00,7.642843e+06,0.0,0.000000
2,0.608345,0.231049,0.009336,0.000000e+00,2.420994e+07,6.855555e+06,0.0,45069.314756
3,1.595427,0.600502,0.025934,0.000000e+00,5.794947e+07,6.111950e+06,30700000.0,0.000000
4,0.132210,0.077016,0.003112,0.000000e+00,7.443250e+07,0.000000e+00,84600000.0,96974.773940
5,0.191451,0.075374,0.003112,1.962856e+07,2.420994e+07,0.000000e+00,30700000.0,92051.480219
6,1.090096,0.641804,0.022478,0.000000e+00,6.616106e+07,0.000000e+00,340700000.0,0.000000
7,0.004392,0.075374,0.003112,7.952400e+07,0.000000e+00,0.000000e+00,30700000.0,52189.609630
8,1.595427,0.628120,0.025934,1.962856e+07,0.000000e+00,6.111950e+06,30700000.0,0.000000
9,0.202782,0.077016,0.003112,0.000000e+00,7.956879e+07,0.000000e+00,0.0,108448.038633


#### Exclude 7 solutions where Number of Deaths exceeds the threshold (from the literature)

In [12]:
deaths_thresh = dict(zip("Expected Number of Deaths(1/3)	Expected Number of Deaths(2/5)	Expected Number of Deaths(4)".split("	"),
                         [0.49659, 0.9921, 0.04]))
deaths_thresh

{'Expected Number of Deaths(1/3)': 0.49659,
 'Expected Number of Deaths(2/5)': 0.9921,
 'Expected Number of Deaths(4)': 0.04}

In [13]:
locs = data.apply(lambda x: (x[0] < deaths_thresh["Expected Number of Deaths(1/3)"])
                  & (x[1] < deaths_thresh["Expected Number of Deaths(2/5)"])
                  & (x[2] < deaths_thresh["Expected Number of Deaths(4)"]), axis=1)
worst_results[locs]

,A.1_DikeIncrease,A.2_DikeIncrease,A.3_DikeIncrease,A.4_DikeIncrease,A.5_DikeIncrease,0_RfR,1_RfR,2_RfR,3_RfR,4_RfR,EWS_DaysToThreat,Expected Number of Deaths(1/3),Expected Number of Deaths(2/5),Expected Number of Deaths(4),Dike Investment Costs(1/3),Dike Investment Costs(2/5),Dike Investment Costs(4),RfR Total Costs,Expected Evacuation Costs
1,6,0,10,3,0,0,0,0,0,0,0,0.000000,0.641804,0.024754,8.122873e+07,0.000000e+00,7.642843e+06,0.0,0.000000
4,0,5,0,0,1,1,0,0,0,0,4,0.132210,0.077016,0.003112,0.000000e+00,7.443250e+07,0.000000e+00,84600000.0,96974.773940
5,0,0,1,0,1,0,0,1,0,0,3,0.191451,0.075374,0.003112,1.962856e+07,2.420994e+07,0.000000e+00,30700000.0,92051.480219
7,7,0,6,0,0,0,0,1,0,0,3,0.004392,0.075374,0.003112,7.952400e+07,0.000000e+00,0.000000e+00,30700000.0,52189.609630
9,0,4,0,0,6,0,0,0,0,0,4,0.202782,0.077016,0.003112,0.000000e+00,7.956879e+07,0.000000e+00,0.0,108448.038633
11,6,4,1,0,1,0,0,0,0,0,4,0.131291,0.077016,0.003112,6.926022e+07,7.085816e+07,0.000000e+00,0.0,96832.037931
13,0,4,0,1,0,0,1,0,1,1,3,0.126259,0.067444,0.002242,0.000000e+00,4.664822e+07,6.111950e+06,595100000.0,77631.425711
14,7,4,3,1,0,0,0,0,0,0,3,0.098435,0.077016,0.003112,7.573571e+07,4.664822e+07,6.111950e+06,0.0,74490.726675
15,0,0,1,0,0,0,0,1,0,1,3,0.190736,0.075374,0.002697,1.962856e+07,0.000000e+00,0.000000e+00,286800000.0,91611.374714


### 3) Possible Thresholds for Costs

#### Cost Ranges of final 9 Solutions

In [14]:
worst_results[locs].iloc[:, -5:]

,Dike Investment Costs(1/3),Dike Investment Costs(2/5),Dike Investment Costs(4),RfR Total Costs,Expected Evacuation Costs
1,8.122873e+07,0.000000e+00,7.642843e+06,0.0,0.000000
4,0.000000e+00,7.443250e+07,0.000000e+00,84600000.0,96974.773940
5,1.962856e+07,2.420994e+07,0.000000e+00,30700000.0,92051.480219
7,7.952400e+07,0.000000e+00,0.000000e+00,30700000.0,52189.609630
9,0.000000e+00,7.956879e+07,0.000000e+00,0.0,108448.038633
11,6.926022e+07,7.085816e+07,0.000000e+00,0.0,96832.037931
13,0.000000e+00,4.664822e+07,6.111950e+06,595100000.0,77631.425711
14,7.573571e+07,4.664822e+07,6.111950e+06,0.0,74490.726675
15,1.962856e+07,0.000000e+00,0.000000e+00,286800000.0,91611.374714


##### Cost

maximum

In [15]:
worst_results[locs].iloc[:, -5:].max()

Dike Investment Costs(1/3)    8.122873e+07
Dike Investment Costs(2/5)    7.956879e+07
Dike Investment Costs(4)      7.642843e+06
RfR Total Costs               5.951000e+08
Expected Evacuation Costs     1.084480e+05
dtype: float64

proportional to the maximum

In [14]:
worst_results[locs].iloc[:, -5:].max() * 0.9

Dike Investment Costs(1/3)    7.310586e+07
Dike Investment Costs(2/5)    7.161191e+07
Dike Investment Costs(4)      6.878559e+06
RfR Total Costs               5.355900e+08
Expected Evacuation Costs     9.760323e+04
dtype: float64

mean value

In [15]:
worst_results[locs].iloc[:, -5:].mean()

Dike Investment Costs(1/3)    3.833398e+07
Dike Investment Costs(2/5)    3.804065e+07
Dike Investment Costs(4)      2.207416e+06
RfR Total Costs               1.142111e+08
Expected Evacuation Costs     7.669216e+04
dtype: float64

##### Deaths

literature

In [16]:
deaths_thresh = pd.Series(deaths_thresh) * 1
deaths_thresh

Expected Number of Deaths(1/3)    0.49659
Expected Number of Deaths(2/5)    0.99210
Expected Number of Deaths(4)      0.04000
dtype: float64

lower it by multiplier

In [17]:
deaths_thresh = pd.Series(deaths_thresh) * 0.5
deaths_thresh

Expected Number of Deaths(1/3)    0.248295
Expected Number of Deaths(2/5)    0.496050
Expected Number of Deaths(4)      0.020000
dtype: float64

zero-death allowance

In [18]:
deaths_thresh = pd.Series(deaths_thresh) * 0
deaths_thresh

Expected Number of Deaths(1/3)    0.0
Expected Number of Deaths(2/5)    0.0
Expected Number of Deaths(4)      0.0
dtype: float64